In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm as tqdm_nb
import copy
import os
import yaml

#from qiskit import aer, transpile, execute
#from qiskit.utils import QuantumInstance
#from qiskit.visualization import *
import qiskit
from qiskit.quantum_info import hellinger_fidelity

import matplotlib
import matplotlib.pyplot as plt

plt.style.use('dark_background')
matplotlib.rcParams['figure.figsize'] = (16, 9)
matplotlib.rcParams.update({'font.size': 12})

plt.rcParams['axes.facecolor'] = '0.2'
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.get_cmap('tab10').colors)

## Cluster

### Load results

In [2]:
sub = 'cluster_model/data/final'
os.listdir(sub)

['2023_07_30_cluster_exp_7qubits_g06_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-001_121500shots_8.yaml',
 '2023_07_30_cluster_exp_7qubits_g-01_121500shots_3.yaml',
 'notes.txt',
 '2023_07_30_cluster_exp_7qubits_g08_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-07_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g09_121500shots_5.yaml',
 '2023_07_30_cluster_exp_7qubits_g01_121500shots_3.yaml',
 '2023_07_30_cluster_exp_7qubits_g-06_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-08_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-09_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g07_121500shots_6.yaml',
 '2023_07_30_cluster_exp_7qubits_g1_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g03_121500shots_3.yaml',
 '2023_07_30_cluster_exp_7qubits_g-04_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-05_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g05_121500shots_4.yaml',
 '2023_07_30_cluster_exp_7qubits_g02_121500shots_1

In [3]:
names = [
'2023_07_30_cluster_exp_7qubits_g-001_121500shots_8.yaml',
 '2023_07_30_cluster_exp_7qubits_g-01_121500shots_3.yaml',
 '2023_07_30_cluster_exp_7qubits_g-02_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-03_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g-04_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-05_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g-06_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-07_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g-08_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g-09_121500shots_2.yaml',
 '2023_07_30_cluster_exp_7qubits_g-1_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g01_121500shots_3.yaml',
 '2023_07_30_cluster_exp_7qubits_g02_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g03_121500shots_3.yaml',
 '2023_07_30_cluster_exp_7qubits_g04_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g05_121500shots_4.yaml',
 '2023_07_30_cluster_exp_7qubits_g06_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g07_121500shots_6.yaml',
 '2023_07_30_cluster_exp_7qubits_g08_121500shots_1.yaml',
 '2023_07_30_cluster_exp_7qubits_g09_121500shots_5.yaml',
 '2023_07_30_cluster_exp_7qubits_g1_121500shots_1.yaml',
]
spaces = []
results = []

for name in names: 
    with open(f'{sub}/' + name, 'r') as file:
        result = yaml.load(file, Loader=yaml.FullLoader)
        results.append(result)

### Comparison with simulation

In [4]:
print(f"{'File':55} | {'Shots':5} | {'Mean':5} | {'Min':5} | {'Max':5} |")

bases = 243
fid_threshold = 0.8
i = 0
good_gs = []
for name, res in zip(names, results):
    if i in spaces:
        print()
    i+=1
    
    fid_mean = np.mean([f['Hellinger fidelity'] for f in res])
    fid_min = np.min([f['Hellinger fidelity'] for f in res])
    fid_max = np.max([f['Hellinger fidelity'] for f in res])

    l = '' if len(res) == bases else len(res)
    if (fid_min > fid_threshold) and len(res) == bases: 
        good = 'Good'
        good_gs.append(res[0]['g'])
    else:
        good = ''
    print(f'{name:55} | {l:5} | {fid_mean:4.3f} | {fid_min:4.3f} | {fid_max:4.3f} | {good:5}')
    

File                                                    | Shots | Mean  | Min   | Max   |
2023_07_30_cluster_exp_7qubits_g-001_121500shots_8.yaml |       | 0.937 | 0.842 | 0.987 | Good 
2023_07_30_cluster_exp_7qubits_g-01_121500shots_3.yaml  |       | 0.943 | 0.860 | 0.981 | Good 
2023_07_30_cluster_exp_7qubits_g-02_121500shots_1.yaml  |       | 0.950 | 0.887 | 0.985 | Good 
2023_07_30_cluster_exp_7qubits_g-03_121500shots_2.yaml  |       | 0.947 | 0.867 | 0.982 | Good 
2023_07_30_cluster_exp_7qubits_g-04_121500shots_1.yaml  |       | 0.950 | 0.890 | 0.983 | Good 
2023_07_30_cluster_exp_7qubits_g-05_121500shots_2.yaml  |       | 0.948 | 0.887 | 0.982 | Good 
2023_07_30_cluster_exp_7qubits_g-06_121500shots_1.yaml  |       | 0.949 | 0.884 | 0.983 | Good 
2023_07_30_cluster_exp_7qubits_g-07_121500shots_2.yaml  |       | 0.950 | 0.851 | 0.983 | Good 
2023_07_30_cluster_exp_7qubits_g-08_121500shots_1.yaml  |       | 0.949 | 0.869 | 0.979 | Good 
2023_07_30_cluster_exp_7qubits_g-09_121500shot

## AKLT

### Load results

In [5]:
sub = 'aklt_model/data-9cnot/final'
os.listdir(sub)

['2023_07_28_aklt_exp_8qubits_g-07_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-06_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g03_128000shots_2.yaml',
 'notes.txt',
 '2023_07_28_aklt_exp_8qubits_g-09_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g02_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g04_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g05_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-01_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-1_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g07_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g06_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g08_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g099_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-03_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-02_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g09_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-04_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g-05_128000sh

In [6]:
names = [
'2023_07_28_aklt_exp_8qubits_g-001_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g-01_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-02_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g-03_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-04_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g-05_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-06_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g-07_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-08_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-09_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g-1_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g01_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g02_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g03_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g04_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g05_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g06_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g07_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g08_128000shots_1.yaml',
 '2023_07_28_aklt_exp_8qubits_g099_128000shots_2.yaml',
 '2023_07_28_aklt_exp_8qubits_g09_128000shots_2.yaml',
]
spaces = []
results = []

for name in names: 
    with open(f'{sub}/' + name, 'r') as file:
        result = yaml.load(file, Loader=yaml.FullLoader)
        results.append(result)

### Comparison with simulation

In [7]:
print(f"{'File':55} | {'Shots':5} | {'Mean':5} | {'Min':5} | {'Max':5} |")

bases = 64
fid_threshold = 0.55
i = 0
for name, res in zip(names, results):
    if i in spaces:
        print()
    i+=1
    
    fid_mean = np.mean([f['Hellinger fidelity'] for f in res])
    fid_min = np.min([f['Hellinger fidelity'] for f in res])
    fid_max = np.max([f['Hellinger fidelity'] for f in res])

    l = '' if len(res) == bases else len(res)
    good = 'Good' if (fid_min > fid_threshold) and len(res) == bases else ''
    print(f'{name:55} | {l:5} | {fid_mean:4.3f} | {fid_min:4.3f} | {fid_max:4.3f} | {good:5}')
    

File                                                    | Shots | Mean  | Min   | Max   |
2023_07_28_aklt_exp_8qubits_g-001_128000shots_1.yaml    |       | 0.803 | 0.742 | 0.860 | Good 
2023_07_28_aklt_exp_8qubits_g-01_128000shots_2.yaml     |       | 0.816 | 0.751 | 0.861 | Good 
2023_07_28_aklt_exp_8qubits_g-02_128000shots_1.yaml     |       | 0.819 | 0.715 | 0.870 | Good 
2023_07_28_aklt_exp_8qubits_g-03_128000shots_2.yaml     |       | 0.812 | 0.747 | 0.871 | Good 
2023_07_28_aklt_exp_8qubits_g-04_128000shots_1.yaml     |       | 0.820 | 0.763 | 0.866 | Good 
2023_07_28_aklt_exp_8qubits_g-05_128000shots_2.yaml     |       | 0.845 | 0.768 | 0.891 | Good 
2023_07_28_aklt_exp_8qubits_g-06_128000shots_1.yaml     |       | 0.831 | 0.777 | 0.873 | Good 
2023_07_28_aklt_exp_8qubits_g-07_128000shots_2.yaml     |       | 0.843 | 0.760 | 0.877 | Good 
2023_07_28_aklt_exp_8qubits_g-08_128000shots_2.yaml     |       | 0.826 | 0.754 | 0.863 | Good 
2023_07_28_aklt_exp_8qubits_g-09_128000shots_2